In [89]:
import os
from dotenv import load_dotenv
import openai
import requests
import json
import time
import logging
from datetime import datetime
import fitz
import tensorflow as tf
import keras
import keras_nlp
import logging
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
import docx 

In [90]:
# Load environment variables
try:
    load_dotenv()
    openai.api_key = os.environ.get("OPENAI_API_KEY")
    if not openai.api_key:
        raise ValueError("OpenAI API key not found in environment variables.") 

    logging.info("Environment variables loaded successfully.")
except Exception as e:
    logging.error(f"Error loading environment variables: {e}")
    raise

In [91]:
# Initialize OpenAI client
client = openai.OpenAI()


In [92]:

model = os.getenv("MODEL_NAME", "gpt-4o-mini")

# Create Contract Checking Assistant
instructions = """
You are a Contract Checking Assistant. Your task is to receive written contracts, check them against specified regulations, identify missing parts, suggest modifications, and provide recommendations for improvement.

When a contract is provided, follow these steps:

1. **Check for Compliance**:
    - Verify that the contract complies with Iran trade rules uploade on fil search tool and other iran regulations.
    - Check compliance with ICC and Incoterms regulations which you may find on internet.
2. **Identify Missing Parts**:
    - Highlight any sections that are missing or incomplete based on regular contract structure.
3. **Suggest Modifications**:
    - Recommend changes to ensure the contract meets all legal and regulatory requirements.
4. **Provide Improvements**:
    - Offer suggestions on how to improve the contract for clarity, fairness, and comprehensiveness.
5. **Identify Weaknesses and Challenges**:
    - Find weaknesses and challenging points in the contract and suggest modifications for these parts.

Your responses should be clear, concise, and professional. Always provide detailed explanations for your suggestions and ensure that your feedback is actionable.
"""

In [93]:
# Create Contract_Checking_Assistant
Contract_Checking_Assistant = client.beta.assistants.create(
    name="Contract_Check_Assistant_alph",
    instructions=instructions,
    model=model,
    tools=[
        {"type": "file_search"},
        {"type": "code_interpreter"},
    ],
    metadata={"project": "Contract Review"}
)

Contract_Checking_Assistant.id

'asst_5h01BYX8k6uEnt4x5KzRrtsg'

In [94]:
# Create a vector store called "The Rules"
vector_store = client.beta.vector_stores.create(name="The Rules")
vector_store.id

'vs_Fs2sbnvls9GZ5NBtStrJvnL2'

In [95]:
# Upload the file to the vector store
def upload_to_vector_store(file_path, vector_store_id):
    try:
        with open(file_path, 'r') as f:
            embeddings = json.load(f)
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        vector_store = client.vector_stores.get(vector_store_id)
        vector_store.upload(embeddings)
        logging.info(f"Embeddings uploaded to vector store {vector_store_id}")
    except Exception as e:
        logging.error(f"Error uploading embeddings to vector store: {e}")
        raise

In [96]:
file_path = './iran_trade_rules.docx'
file_stream = open(file_path, "rb")
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=[file_stream]
)
file_stream.close()

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [97]:
# %%
# Update the assistant with the vector store
Contract_Checking_Assistant = client.beta.assistants.update(
    assistant_id=Contract_Checking_Assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

In [98]:
# Create a sample thread with a non-empty message content
thread = client.beta.threads.create()
thread.id

'thread_YL4qUF9qDLylzVvEKc9mKUjB'

In [99]:
Contract_Checking_Assistant = client.beta.assistants.update(
    assistant_id=Contract_Checking_Assistant.id,
    tools=[
        {"type": "file_search"},
        {
            "type": "function",
            "function": {
                "name": "contract_check",
                "description": "Check the contract against specified regulations.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "contract_text": {
                            "type": "string",
                            "description": "The text of the contract to be checked."
                        },
                        "regulations": {
                            "type": "array",
                            "items": {
                                "type": "string"
                            },
                            "description": "A list of regulations to check against."
                        }
                    },
                    "required": ["contract_text", "regulations"]
                }
            }
        }
    ]
)
